### Data
Downloaded from: https://www.kaggle.com/divyansh22/flight-delay-prediction

### Question 1.
Among all pairs of ORIGIN and DEST, with which
1. arrival is delayed (i.e., ARR_DEL15 == 1) at least once in both 2019 and 2020,
2. the delayed flight arrives before noon (i.e., ARR_TIME < 1200) in 2019 and after noon (i.e., ARR_TIME >= 1200) in 2020,

how many uique pairs of ORIGIN and DEST that such delays have occurred do we have?

### 문제 1.
2019년과 2020년 데이터의 ORIGIN과 DEST간 비행 기록 중에서,
1. 2019년과 2020년 모두 비행기 연착이 발생한 적이 있고
2. 또 그 연착된 비행기가 2019년엔 정오 이전에 도착하고 2020년엔 정오 이후에 도착한,

연착 기록을 찾아 그러한 연착이 일어난 ORIGIN과 DEST 구간의 가지수를 계산하시오.


In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master("local") \
        .appName("Final quiz 1") \
        .config("spark.ui.port", "4050") \
        .getOrCreate()

sc = spark.sparkContext

22/11/10 16:09:46 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


(1) 아래 셀에 Jan_2019_ontime.csv 파일을 읽어 data2019 라는 이름의 dataframe을 생성하시오. 같은 식으로 Jan_2020_ontime.csv 파일을 읽어 data2020 라는 이름의 dataframe을 생성하시오. (10점)

In [56]:
from pyspark.sql.types import *

schema = StructType([
    StructField('DAY_OF_MONTH', IntegerType(), True),
    StructField('DAY_OF_WEEK', IntegerType(), True),
    StructField('OP_UNIQUE_CARRIER', StringType(), True),
    StructField('OP_CARRIER_AIRLINE_ID', IntegerType(), True),
    StructField('OP_CARRIER', StringType(), True),
    StructField('TAIL_NUM', StringType(), True),
    StructField('OP_CARRIER_FL_NUM', IntegerType(), True),
    StructField('ORIGIN_AIRPORT_ID', IntegerType(), True),
    StructField('ORIGIN_AIRPORT_SEQ_ID', IntegerType(), True),
    StructField('ORIGIN', StringType(), True),
    StructField('DEST_AIRPORT_ID', IntegerType(), True),
    StructField('DEST_AIRPORT_SEQ_ID', IntegerType(), True),
    StructField('DEST', StringType(), True),
    StructField('DEP_TIME', StringType(), True),
    StructField('DEP_DEL15', DoubleType(), True),
    StructField('DEP_TIME_BLK', StringType(), True),
    StructField('ARR_TIME', StringType(), True),
    StructField('ARR_DEL15', DoubleType(), True),
    StructField('CANCELLED', DoubleType(), True),
    StructField('DIVERTED', DoubleType(), True),
    StructField('DISTANCE', DoubleType(), True),
])

data2019 = spark.read.format('csv')\
                     .option('header', 'true')\
                     .option("quote", "\"")\
                     .option("escape", "\"")\
                     .option("multiLine", "true")\
                     .schema(schema)\
                     .load('../../data/final_quiz/Jan_2019_ontime.csv')

data2020 = spark.read.format('csv')\
                     .option('header', 'true')\
                     .option("quote", "\"")\
                     .option("escape", "\"")\
                     .option("multiLine", "true")\
                     .schema(schema)\
                     .load('../../data/final_quiz/Jan_2020_ontime.csv')

(2) 아래 셀에 data2019 에 결측값이 있는지 확인하는 코드를 작성하시오 (즉, 결측값의 개수를 출력하시오). 같은 식으로 아래 셀에 data2020 에 결측값이 있는지 확인하는 코드를 작성하시오 (10점)

In [57]:
from pyspark.sql.functions import col,isnan,when,count
print('2019')
data2019.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data2019.columns]).show()
print('2020')
data2020.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data2020.columns]).show()

2019


+------------+-----------+-----------------+---------------------+----------+--------+-----------------+-----------------+---------------------+------+---------------+-------------------+----+--------+---------+------------+--------+---------+---------+--------+--------+
|DAY_OF_MONTH|DAY_OF_WEEK|OP_UNIQUE_CARRIER|OP_CARRIER_AIRLINE_ID|OP_CARRIER|TAIL_NUM|OP_CARRIER_FL_NUM|ORIGIN_AIRPORT_ID|ORIGIN_AIRPORT_SEQ_ID|ORIGIN|DEST_AIRPORT_ID|DEST_AIRPORT_SEQ_ID|DEST|DEP_TIME|DEP_DEL15|DEP_TIME_BLK|ARR_TIME|ARR_DEL15|CANCELLED|DIVERTED|DISTANCE|
+------------+-----------+-----------------+---------------------+----------+--------+-----------------+-----------------+---------------------+------+---------------+-------------------+----+--------+---------+------------+--------+---------+---------+--------+--------+
|           0|          0|                0|                    0|         0|    2543|                0|                0|                    0|     0|              0|                 

+------------+-----------+-----------------+---------------------+----------+--------+-----------------+-----------------+---------------------+------+---------------+-------------------+----+--------+---------+------------+--------+---------+---------+--------+--------+
|DAY_OF_MONTH|DAY_OF_WEEK|OP_UNIQUE_CARRIER|OP_CARRIER_AIRLINE_ID|OP_CARRIER|TAIL_NUM|OP_CARRIER_FL_NUM|ORIGIN_AIRPORT_ID|ORIGIN_AIRPORT_SEQ_ID|ORIGIN|DEST_AIRPORT_ID|DEST_AIRPORT_SEQ_ID|DEST|DEP_TIME|DEP_DEL15|DEP_TIME_BLK|ARR_TIME|ARR_DEL15|CANCELLED|DIVERTED|DISTANCE|
+------------+-----------+-----------------+---------------------+----------+--------+-----------------+-----------------+---------------------+------+---------------+-------------------+----+--------+---------+------------+--------+---------+---------+--------+--------+
|           0|          0|                0|                    0|         0|     698|                0|                0|                    0|     0|              0|                 

(3) 아래 셀에 data2019 의 row 중에서 도착지연이 된 row를 담은 delay2019란 dataframe을 생성하시오. 같은 식으로 data2020 의 row 중에서 도착지연이 된 row를 담은 delay2020란 dataframe을 생성하시오. (5점) 

In [58]:
delay2019 = data2019.filter(data2019.ARR_DEL15 == 1.00)
delay2020 = data2020.filter(data2020.ARR_DEL15 == 1.00)

(4) 아래 셀에 delay2019 의 row 중에서 도착시간이 정오(1200) 보다 이른 (<) row 를 담은 dataframe인 변수 이름 df1를 생성하시오. 또한 같은 식으로 delay2020 의 row 중에서 도착시간이 정오(1200) 보다 늦은 (>=) row 를 담은 dataframe인 변수 이름 df2를 생성하시오. (5점) 

In [59]:
df1 = delay2019.filter(data2019.ARR_TIME < '1200')
df2 = delay2020.filter(data2020.ARR_TIME >= '1200')

(5) 아래 셀에 df1과 df2 의 row 중에서 ORIGIN과 DEST가 일치하는 row만 남겨 변수명 result의 dataframe을 생성하시오.  (10점)

In [60]:
cond = [df1.ORIGIN == df2.ORIGIN, df1.DEST == df2.DEST]
result = df1.join(df2, cond).select(df1.ORIGIN, df1.DEST)
result.show(5)

+------+----+
|ORIGIN|DEST|
+------+----+
|   PHL| MCO|
|   PHL| MCO|
|   PHL| MCO|
|   PHL| MCO|
|   PHL| MCO|
+------+----+
only showing top 5 rows



(6) 아래 셀에 result의 ORIGIN과 DEST의 쌍의 가짓수 (즉, distinct한 쌍의 개수)를 세어 출력하시오. (10점)

In [61]:
result.distinct().count()

2651

### Question 2.
Among all flight data in Jan 2019 and Jan 2020 (I mean union of them), what is is maximum numbers that an airline company committed arrival delays in a single days?

### 문제 2.
2019년1월과 2020년1월 데이터의 ORIGIN과 DEST간 비행 기록 중에서, 하루에 한 항공사가 일으킨 도착지연 횟수중에서 최대치는?

(7) df1이란 변수명으로 delay2019에 YEAR라는 이름의 column을 추가하고 값을 숫자 2019로 채워 넣은 dataframe 을 생성하시오. 같은 식으로 df2란 변수명에는 delay2020에 YEAR라는 이름의 column을 추가하고 값을 숫자 2020로 채워 넣은 dataframe 을 생성하시오. (10점)

In [29]:
from pyspark.sql.functions import col,lit
df1 = delay2019.withColumn('YEAR', lit(2019))
df2 = delay2020.withColumn('YEAR', lit(2020))

(8) df1과 df2의 RDD를 각각 rdd1과 rdd2란 변수에 대입하시오.(5점)

In [31]:
rdd1, rdd2 = df1.rdd, df2.rdd

(9) rdd1과 rdd2를 합친 RDD rdd3을 생성하시오 (5점)

In [38]:
rdd3 = rdd1.union(rdd2)

(10) 이제 문제 2를 풀기 위한 아이디어는 다음과 같다.

1. "DAY_OF_MONTH", "YEAR", "OP_UNIQUE_CARRIER" 이 세 값에 대해 rdd3에 나타나는 횟수를 센다 (즉, 각 항공사별로 그리고 각 날짜별로 delay를 일으킨 횟수를 센다)
2. 여러가지 방법이 있지만 우리는 "DAY_OF_MONTH", "YEAR", "OP_UNIQUE_CARRIER" 세 값으로 이루어진 튜플로 reduceByKey를 이용해 그 출연횟수를 세기로 한다.
3. 그러기 위해 우선 rdd3의 row로 부터 키-값의 쌍 형태의 RDD를 적절히 생성한다.
4. reduceByKey를 이용해 카운트를 하고 map (transformation)과 max (action)를 이용해 답을 구한다. 

우선, 아래 셀에서 Row로 이루어진 rdd3을 적절히 transformation하여 키-값의 쌍 형식의 RDD rdd4를 만든다. (10점)

In [44]:
rdd4 = rdd3.map(lambda x: ((x[0], x[-1], x[2]), 1))

In [45]:
rdd4.take(5)

[((1, 2019, '9E'), 1),
 ((1, 2019, '9E'), 1),
 ((1, 2019, '9E'), 1),
 ((1, 2019, '9E'), 1),
 ((1, 2019, '9E'), 1)]

(11) "DAY_OF_MONTH", "YEAR", "OP_UNIQUE_CARRIER" 세 값으로 이루어진 튜플의 출연횟수를 reduceByKey를 이용해 count한다. 그 결과는 rdd5에 담는다. (10점)

In [50]:
rdd5 = rdd4.reduceByKey(lambda x, y: x + y).map(lambda x: x[1])

(12) 출현 횟수중 최대값을 출력한다 (10점)

In [52]:
rdd5.max()

1340

In [62]:
sc.stop()